# DuplicateBookModal E2E Accessibility Test

This notebook uses Playwright and axe-core to perform an end-to-end accessibility audit of the DuplicateBookModal component.

## 1. Import Required Libraries and Setup

Install and import Playwright and axe-core for E2E and accessibility testing.

In [ ]:
# Install Playwright and axe-core if not already installed
!pip install playwright
!playwright install
!pip install axe-selenium-python

## 2. Launch Application and Navigate to DuplicateBookModal

This cell launches the app and navigates to a state where the DuplicateBookModal is visible. Ensure your app has a button with `data-testid="open-duplicate-book-modal"` for test automation.

In [ ]:
from playwright.sync_api import sync_playwright
APP_URL = "http://localhost:3000"  # Update if your dev server runs elsewhere

def open_duplicate_book_modal():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        page.goto(APP_URL)
        # Trigger the DuplicateBookModal (ensure a button with this test id exists in your app)
        page.click('[data-testid="open-duplicate-book-modal"]')
        return browser, page

browser, page = open_duplicate_book_modal()
print("DuplicateBookModal opened.")

## 3. Interact with DuplicateBookModal Using Keyboard Navigation

This cell simulates keyboard navigation (Tab, Shift+Tab, Enter, Escape) to ensure all interactive elements are accessible.

In [ ]:
# Tab through all focusable elements in the modal and print their accessible names
dialog = page.get_by_role('dialog')
focusable = dialog.query_selector_all('button, [href], input, select, textarea, [tabindex]:not([tabindex="-1"])')
assert len(focusable) > 0, "There should be at least one focusable element in the DuplicateBookModal"

for i, el in enumerate(focusable):
    if i == 0:
        # First element should be focused initially
        assert page.evaluate('document.activeElement === arguments[0]', el), f"Element {i} should be focused initially"
    else:
        page.keyboard.press('Tab')
        assert page.evaluate('document.activeElement === arguments[0]', el), f"Element {i} should be focused after Tab"
    print(f"Tab {i}:", el.inner_text())

## 4. Verify ARIA Attributes and Roles

This cell checks that the modal and its elements have correct ARIA roles, labels, and properties for accessibility.

In [ ]:
# Check ARIA roles and labels
assert dialog.get_attribute('role') == 'dialog', "Dialog should have role='dialog'"
assert dialog.get_attribute('aria-modal') == 'true', "Dialog should have aria-modal='true'"
heading = dialog.query_selector('h2')
assert heading is not None and 'Duplicate Book Found' in heading.inner_text(), "Heading should be present and correct"
close_btn = dialog.query_selector('button[aria-label="Close"]')
assert close_btn is not None, "Close button with aria-label should be present"

## 5. Test Screen Reader Output

This cell uses axe-core to check that screen readers announce the modal and its content appropriately.

In [ ]:
# Inject and run axe-core for accessibility analysis
import requests
axe_url = "https://cdnjs.cloudflare.com/ajax/libs/axe-core/4.7.2/axe.min.js"
axe_script = requests.get(axe_url).text
page.add_script_tag(content=axe_script)
results = page.evaluate("async () => await axe.run(document.querySelector('[role=dialog]'))")
violations = results['violations']
if violations:
    print("Axe-core found accessibility violations:")
    for v in violations:
        print(f"- {v['id']}: {v['description']}")
else:
    print("No accessibility violations found by axe-core.")

## 6. Check Focus Management and Trap

This cell ensures that focus is trapped within the modal and returns to the triggering element when the modal closes.

In [ ]:
# Tab past the last focusable element and ensure focus wraps to the first
page.keyboard.press('Tab')
first_focusable = focusable[0]
assert page.evaluate('document.activeElement === arguments[0]', first_focusable), "Focus should wrap to first element after last"
# Close the modal and check focus returns to trigger
page.keyboard.press('Escape')
trigger = page.query_selector('[data-testid="open-duplicate-book-modal"]')
assert page.evaluate('document.activeElement === arguments[0]', trigger), "Focus should return to trigger after modal closes"

## 7. Validate Color Contrast and Visual Accessibility

This cell checks that text and interactive elements meet WCAG color contrast requirements using axe-core.

In [ ]:
# Check for color contrast issues using axe-core results
contrast_violations = [v for v in violations if v['id'] == 'color-contrast']
if contrast_violations:
    print("Color contrast issues found:")
    for v in contrast_violations:
        print(f"- {v['description']}")
else:
    print("No color contrast issues found.")

## 8. Automate Accessibility Assertions

This cell summarizes the results and asserts that there are no critical accessibility violations.

In [ ]:
# Assert no critical accessibility violations
critical_violations = [v for v in violations if v['impact'] == 'critical']
if critical_violations:
    print("Critical accessibility violations found:")
    for v in critical_violations:
        print(f"- {v['id']}: {v['description']}")
    raise AssertionError("Critical accessibility violations detected!")
else:
    print("No critical accessibility violations detected. Accessibility test passed.")
browser.close()